# Implementing a Architecture

[![Open in Colab](https://img.shields.io/badge/Open%20in%20Colab-F9AB00?style=for-the-badge&logo=googlecolab&color=525252)](https://colab.research.google.com/github/H-IAAC/CST-Python/blob/main/examples/Implementing%20a%20Architecture.ipynb) [![Open in Github](https://img.shields.io/badge/Open%20in%20Github-100000?style=for-the-badge&logo=github&logoColor=white)](https://github.com/H-IAAC/CST-Python/blob/main/examples/Implementing%20a%20Architecture.ipynb)

A cognitive architecture in the CST is implemented using a combination of Codelets and Memories inside a Mind. Each Codelet will communicate with the others using only the Memories.

## Diagrams

Before implementing the code, it is common to prepare a diagram describing all the elements of the architecture.

For that, it is defined a visual language:

The **Codelet** symbol:

![](./Implementing%20a%20Architecture/Codelet.png)

The string inside the symbol is the Codelet name. The left ports are its inputs, the _local input (LI)_ and the _global input (GI)_, and the right ports its outputs, the _standard output (O)_ and the _activation level (A)_. The LI and A ports will be discussed in later examples.

The Memory Object symbol:

![](./Implementing%20a%20Architecture/MemoryObject.png)

So, if we have a Codelet "My Codelet", that reads from "Input Memory" and writes to "Output Memory", the diagram is going to be:

![](./Implementing%20a%20Architecture/SimpleDiagram.png)

Its also possible to group memories in a _Memory Group_:

![](./Implementing%20a%20Architecture/MemoryGroup.png)

For creating a diagram, a [draw.io](https://www.drawio.com/) shape library is provided: [shape library](https://github.com/H-IAAC/CST-Python/blob/2065cb3f29931867de37ec39b40839d82c1331fc/resources/CST-Library.xml).

## Example architecture

Our example architecture is going to solve a simple quadratic equation in the form $ax^2+bx+c=0$.

For that, the equation parameters $a$, $b$ and $c$ will be stored in Memory Objects, two "SolveEquation" will solve the equation computing the two different solutions, and a final "JoinResults" codelet will join each result to a final memory:

![](./Implementing%20a%20Architecture/diagram.png)

For that, we start importing the necessary modules:

In [1]:
try:
    import cst_python
except:
    !python3 -m pip install cst_python

In [2]:
import math # Math operations
import time # Sleep

import cst_python as cst # CST-Python module

And we can implement the `SolveEquationCodelet` using the quadratic formula. Note that the codelet can receive two parameters: if should compute the second solution (`negative`), and the output memory name `output_name`:

In [3]:
class SolveEquationCodelet(cst.Codelet):

    def __init__(self, negative:bool=False, output_name:str="x"):
        super().__init__()

        self._negative = negative
        self._output_name = output_name

        self._a_mo : None | cst.MemoryObject = None
        self._b_mo : None | cst.MemoryObject = None
        self._c_mo : None | cst.MemoryObject = None
        self._x_mo : None | cst.MemoryObject = None

    def access_memory_objects(self):
        self._a_mo = self.get_input(name="a")
        self._b_mo = self.get_input(name="b")
        self._c_mo = self.get_input(name="c")

        self._x_mo = self.get_output(name=self._output_name)

    def calculate_activation(self):
        pass

    def proc(self):
        a = self._a_mo.get_info()
        b = self._b_mo.get_info()
        c = self._c_mo.get_info()

        delta = math.pow(b, 2) - (4*a*c)
        delta_sqrt = math.sqrt(delta)

        if self._negative:
            delta_sqrt *= -1
        
        if a == 0:
            if b == 0:
                x = float("nan")
            else:
                x = -c/b
        else:  
            x = (-b + delta_sqrt)/(2*a)

        self._x_mo.set_info(x)
    

The `JoinResultsCodelet` is going to get all its inputs and send to a unified result:

In [4]:
class JoinResultsCodelet(cst.Codelet):
    def __init__(self):
        super().__init__()
        self._result_mo : None | cst.MemoryObject = None

    def access_memory_objects(self):
        self._result_mo = self.get_output(name="result")

    def calculate_activation(self):
        pass

    def proc(self):
        result = []

        for input_mo in self.inputs:
            result.append(input_mo.get_info())

        self._result_mo.set_info(result)


Now that we have all the codelets defined, we can start creating the agent mind:

In [5]:
mind = cst.Mind()

Starting with the memories, observe that the Memory Groups defined in the diagram is created using a `mind.create_memory_group`, and each memory is assigned to the group using `register_memory`, and that each memory starting info is defined (instead of `None`) to avoid exceptions inside the codelets:

In [6]:
# Input

mind.create_memory_group("Input")

a_mo = mind.create_memory_object("a", 0)
b_mo = mind.create_memory_object("b", 0)
c_mo = mind.create_memory_object("c", 0)

mind.register_memory(a_mo, "Input")
mind.register_memory(b_mo, "Input")
mind.register_memory(c_mo, "Input")

# Working Memory

mind.create_memory_group("Working Memory")

x1_mo = mind.create_memory_object("x1", 0)
x2_mo = mind.create_memory_object("x2", 0)

mind.register_memory(x1_mo, "Working Memory")
mind.register_memory(x2_mo, "Working Memory")

# Output

mind.create_memory_group("Output")

result_mo = mind.create_memory_object("result")

mind.register_memory(result_mo, "Output")


We than create the SolveEquationCodelets:

In [7]:
solve_equation1 = SolveEquationCodelet(output_name="x1")
solve_equation2 = SolveEquationCodelet(output_name="x2", negative=True)

solve_equation1.add_inputs([a_mo, b_mo, c_mo])
solve_equation2.add_inputs([a_mo, b_mo, c_mo])

solve_equation1.add_output(x1_mo)
solve_equation2.add_output(x2_mo)

And the JoinResultsCodelet:

In [8]:
join_results = JoinResultsCodelet()

join_results.add_inputs([x1_mo, x2_mo])
join_results.add_output(result_mo)

Finally, we can add the codelets to the mind and start it. Before that, we change the time step of each codelet to 10 ms:

In [9]:
solve_equation1.time_step = 10
solve_equation2.time_step = 10
join_results.time_step = 10

mind.insert_codelet(solve_equation1)
mind.insert_codelet(solve_equation2)
mind.insert_codelet(join_results)

mind.start()

We can check the initial result for the equation $0x^2+0x+c = 0$, that is unsovable equation: 

In [10]:
result_mo.get_info()

[nan, nan]

We can than change the input values to solve different equations:

$1x^2 = 0$

In [11]:
a_mo.set_info(1)

time.sleep(0.1)

result_mo.get_info()

[0.0, 0.0]

$-4x^2+8x+0=0$

In [12]:
a_mo.set_info(-4)
b_mo.set_info(8)
c_mo.set_info(0)

time.sleep(0.1)

result_mo.get_info()

[-0.0, 2.0]

$5x^2-6x+9=0$

In [13]:
a_mo.set_info(1)
b_mo.set_info(-6)
c_mo.set_info(9)

time.sleep(0.1)

result_mo.get_info()

[3.0, 3.0]

In the end, we stop the agent mind:

In [14]:
mind.shutdown()